# 1. Preprocess the MODIS GLASS net radiation

### 1.1 Download the data from https://www.glass.hku.hk/archive/RN/MODIS/
### !Note: url is RN not NR

In [ ]:
#!/usr/bin/env python3
from pathlib import Path
import subprocess

OUT_DIR = Path("/share/home/dq076/bedrock/data/NR/GLASS")
OUT_DIR.mkdir(parents=True, exist_ok=True)

for year in range(2003, 2021):
    url = f"https://www.glass.hku.hk/archive/RN/MODIS/{year}/"
    print(f"[YEAR] {year}")

    cmd = [
        "wget",
        "-r",                      # 递归
        "-np",                     # 不进入上级目录
        "-nH",                     # 不创建域名目录
        "--cut-dirs=5",            # 去掉 archive/DSR/MODIS/0.05D/{year}
        "-A", "*.hdf",             # 只下 hdf
        "-c",                      # 断点续传
        "--tries=0",               # 无限重试
        "--retry-connrefused",
        "--waitretry=5",
        "--timeout=30",
        "-P", str(OUT_DIR),        # 输出目录
        url,
    ]

    subprocess.run(cmd, check=True)
